In [41]:
import numpy as np
import pandas as pd
import json
import re
#must pip install textstat
import textstat

Begin by reading in reddit json data

In [3]:
with open('/Users/jayshenoy/Documents/DataScienceLabs/Final_Project/RC_2018-02-01.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 
frame=pd.DataFrame(data)
frame.head()

,author,author_cakeday,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,...,is_submitter,link_id,parent_id,permalink,retrieved_on,score,stickied,subreddit,subreddit_id,subreddit_type
0,YouthfulPhotographer,NaN,None,None,Welcome to generation void,True,0,1517443200,None,False,...,False,t3_7uaobc,t1_dtj8323,/r/tumblr/comments/7uaobc/nihilism_across_gene...,1518478113,7,False,tumblr,t5_2r7hk,public
1,jasonklacour,NaN,None,None,Welcome,True,0,1517443200,None,False,...,False,t3_7ude45,t3_7ude45,/r/pics/comments/7ude45/7_years_later_im_offic...,1518478113,1,False,pics,t5_2qh0u,public
2,Assassin2000,NaN,None,None,I'm 16 and the friend told me he was joking af...,True,0,1517443200,None,False,...,True,t3_7uegdc,t1_dtjo9qq,/r/legaladvice/comments/7uegdc/took_a_awful_jo...,1518478113,1,False,legaladvice,t5_2rawz,public
3,kawaiicicle,NaN,Employee,Assistant Manager,What? \nIt’s a niche rpg. Most rpg fans are ad...,True,0,1517443200,None,False,...,False,t3_7u7mps,t1_dtin0yf,/r/GameStop/comments/7u7mps/is_this_possible/d...,1518478113,0,False,GameStop,t5_2spz2,public
4,recklessmaterialism,NaN,None,None,solid!,True,0,1517443200,None,False,...,True,t3_7udrqf,t3_7udrqf,/r/Seattle/comments/7udrqf/in_town_for_only_a_...,1518478113,-1,False,Seattle,t5_2qhad,public


In [4]:
frame.columns.values

array(['author', 'author_cakeday', 'author_flair_css_class',
       'author_flair_text', 'body', 'can_gild', 'controversiality',
       'created_utc', 'distinguished', 'edited', 'gilded', 'id',
       'is_submitter', 'link_id', 'parent_id', 'permalink',
       'retrieved_on', 'score', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_type'], dtype=object)

In [5]:
frame.describe()

,controversiality,created_utc,gilded,retrieved_on,score
count,3.194211e+06,3.194211e+06,3.194211e+06,3.194211e+06,3.194211e+06
mean,2.043509e-02,1.517490e+09,1.984841e-04,1.518807e+09,8.892290e+00
std,1.414832e-01,2.646685e+04,1.729891e-02,7.827520e+04,1.245221e+02
min,0.000000e+00,1.517443e+09,0.000000e+00,1.518478e+09,-8.660000e+02
25%,0.000000e+00,1.517464e+09,0.000000e+00,1.518816e+09,1.000000e+00
50%,0.000000e+00,1.517497e+09,0.000000e+00,1.518825e+09,2.000000e+00
75%,0.000000e+00,1.517513e+09,0.000000e+00,1.518833e+09,4.000000e+00
max,1.000000e+00,1.517530e+09,1.200000e+01,1.518842e+09,4.832700e+04


author_cakeday,id, link_id,retrieved_on,subreddit_id

In [6]:
#frame=frame.drop(columns=['author_cakeday','id','link_id','retrieved_on','subreddit_id'])
frame=frame.drop(columns='parent_id')

In [36]:
frame.to_csv("clean_reddit_02_01_18.csv",index=False)

In [38]:
df=pd.read_csv("clean_reddit_02_01_18.csv")
df.head()

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5,6,9,10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,is_submitter,permalink,score,stickied,subreddit,subreddit_type
0,YouthfulPhotographer,NaN,NaN,Welcome to generation void,True,0,1.51744e+09,NaN,False,0,False,/r/tumblr/comments/7uaobc/nihilism_across_gene...,7,False,tumblr,public
1,jasonklacour,NaN,NaN,Welcome,True,0,1.51744e+09,NaN,False,0,False,/r/pics/comments/7ude45/7_years_later_im_offic...,1,False,pics,public
2,Assassin2000,NaN,NaN,I'm 16 and the friend told me he was joking af...,True,0,1.51744e+09,NaN,False,0,True,/r/legaladvice/comments/7uegdc/took_a_awful_jo...,1,False,legaladvice,public
3,kawaiicicle,Employee,Assistant Manager,What? \nIt’s a niche rpg. Most rpg fans are ad...,True,0,1.51744e+09,NaN,False,0,False,/r/GameStop/comments/7u7mps/is_this_possible/d...,0,False,GameStop,public
4,recklessmaterialism,NaN,NaN,solid!,True,0,1.51744e+09,NaN,False,0,True,/r/Seattle/comments/7udrqf/in_town_for_only_a_...,-1,False,Seattle,public


In [24]:
#generating flesch_kincaid_readability
#vocabulary difficulty
def generatefkReadability(frame):
    comments=frame.body
    textrating=[]
    for comment in comments:
        textrating.append(textstat.flesch_kincaid_grade(comment))
    #textstat.flesch_reading_ease(test_data)
    #textstat.smog_index(test_data)
    #textstat.flesch_kincaid_grade(test_data)
    #textstat.coleman_liau_index(test_data)
    #textstat.automated_readability_index(test_data)
    #textstat.dale_chall_readability_score(test_data)
    #textstat.difficult_words(test_data)
    #textstat.linsear_write_formula(test_data)
    #textstat.gunning_fog(test_data)
    #textstat.text_standard(test_data)

    return textrating


In [28]:
#for length of comment
def wordLengthGenerator(frame):
    comments=frame.body
    lengths=[]
    for comment in comments:
        lengths.append(len(comment))
    return lengths



In [27]:
#returns the ratio of uppercase to lower case letters
#returns 0 if length is zero
def upperCaseGenerator(frame):
    comments=frame.body
    ratios=[]
    for comment in comments:
        count = 0
        for c in comment:
            if(c.isupper()):
                count = count + 1
        if(len(comment)>0):
            ratioval=float(count/len(comment))
        else:
            ratioval=0
        ratios.append(ratioval)
    return ratios



In [31]:
#counts number of questions in a comment
#by counting number of question marks
def numberOfQuestionsInAComment(frame):
    comments=frame.body
    questions=[]
    for comment in comments:
        count=0
        for c in comment:
            if(c=='?'):
                count=count+1
        questions.append(count)
    return questions


In [38]:
#naive method of determining whether or
#not the title is a question
def isTitleAQuestion(frame):
    START_WORDS = ["who", "what", "when", "where", "why", "how", "is", "can", "does", "do",
                  "could","should","would","which","whose","whom","are",]
    permalinks=frame.permalink
    isQuestion=[]
    for permalink in permalinks:
        splitty=permalink.split('/')
        title=splitty[5]
        wordsInTitle=title.split('_')
        firstWord=wordsInTitle[0]
        if(firstWord in START_WORDS):
            isQuestion.append(1)
        else:
            isQuestion.append(0)
    
    return isQuestion


[0]